# Transforming Data in Azure Data Pipelines

## Overview
1. Trong Data Factory, tạo 2 Dataflows, mđ là load data từ Dataset SQL DB vào Dataset Synapse
2. Trong Synapse Studio, tạo 1 table sẽ chứa aggregated data
3. Trong Data Factory, tạo 1 Dataset nữa sẽ chứa aggregated data -> Tạo 1 Dataflow nữa, vẫn load data từ Dataset SQL DB, nhưng thêm filter và aggregated column, rồi lưu vào Dataset vừa tạo
4. Tạo Pipeline chỉ chứa Dataflow ở step 3 -> Chạy Pipeline -> Vào Synapse kiểm tra
5. Tạo Pipeline nữa chứa 2 Datàlows ở step 1 -> Chạy Pipeline -> Vào Synapse kiểm tra

## 1. Mapping Dataflows
1. Vào **Data Factory** -> **Author** -> **Data flows** -> **New data flow** --> **Add source** \
  Đặt name là `sourcetablesalesorderheader`, chọn Dataset là `db_sqldb_salesorderheader`

   <img src="images/azure1.png" width=1000>

2. Bật **Data flow debug** -> Đặt **Debug time to live** là "2 hours" -> **OK**

3. Ấn **Debug Settings**, đặt **Row litmit** là `100` -> **Save**

4. Ấn vào dấu `+`, chọn **Sink** -> Đặt name là `sinksynapsesalesordertable`, chọn Dataset là `ds_synapse_salesorderheader`

   <img src="images/azure2.png" width=700>
   <img src="images/azure3.png" width=1000>

   Vào table **Mapping**, có thể bỏ tick "Auto mapping" để kiểm tra. Vào tab **Data preview** để xem table.

5. Trong panel **Properties** bên phải, đặt Name là "dataflow load sales order header to synapse"

5. Làm lại với Source có name là `sourcesqldbcustomer`, Dataset là `db_sqldb_customer`; Sink có name là `sinksynapsecustomer`, Dataset là `ds_synapse_customer` \
   Đặt Name của Data flow là "dataflow load customer to synapse"

   <img src="images/azure4.png" width=1000>

6. **Publish all** -> **Publish**

   <img src="images/azure5.png" width=800>

## 2. Transform and Aggregate Data
1. Vào **Synapse Studio** -> **Develop** -> **New SQL script** -> Chọn Dedicated pool đã tạo -> Chạy
    ```sql
    CREATE TABLE SalesAggregate(
        CustomerID int NOT NULL,
        TotalSales  float
    )
    ```

2. Quay lại **Data Factory** -> **Author** -> Tạo 1 Dataset mới từ **Azure Synapse Analytics**, Name là `ds_synapse_salesaggregatetable`, chọn Table `dbo.SalesAggregate`

3. Tạo một **Data flow** nữa, chọn **Source** giống Section 1 và thêm 1 **Filter**.

   <img src="images/azure6.png" width=1000>

4. Trong **Filter on**, ấn **Open expression builder** -> Điền `SalesOrder!=71774` -> **Save and finish**

   <img src="images/azure7.png" width=1500>

5. Thêm một **Derived Column** -> Điền Column là `TotalSales` -> Mở expression builder -> Điền `SubTotal+TaxAmt+Freight` -> **Save and finish**

   <img src="images/azure8.png" width=1500>

   <img src="images/azure9.png" width=1000>

6. Thêm một **Sink** -> Chọn Dataset là `ds_synapse_salesaggregatetable`.

   <img src="images/azure10.png" width=1000>

7. Đặt Name cho Data flow là "dataflow aggregate sales" -> **Publish all** -> **Publish**

   <img src="images/azure11.png" width=1000>

## 3. Create Pipeline Activity
1. Vào **Pipelines** -> **New pipeline** -> **Move & Transform**. Kéo thả **Data flow** vào.

2. Ấn vào Data flow đó -> **Setings** -> Chọn Data flow là "dataflow aggregate sales", **Staging linked service** là `ls_lake_storage` \
   Trng **Staging storage folder**, ấn **Browse** -> Chọn directory trong container đã tạo. -> **OK**
   <img src="images/azure12.png" width=1000>

3. **Validate** -> **Publish all** -> **Publish** -> **Add trigger** -> **Trigger now** -> **OK* \
   Vào **Monitor** -> **Pipeline runs** -> Ấn vào Pipeline đang chạy -> Ấn vào Data flow 2 lần

   <img src="images/azure13.png" width=1500>

4. Quay lại **Synapse Studio** chạy SQL để kiểm tra
   ```sql
   SELECT TOP 100 * FROM SalesAggregate
   ```

   <img src="images/azure14.png" width=1000>

## 4. Debug, Trigger and Monitor Pipelines
1. Tạo một pipeline khác nối hai Data flow với nhau, đặt Name là "Data flow - transfer sales order header to synapse" và "Data flow - transfer customer to synapse". \
   Chọn **Staging storage folder** như ở Sec 3

    <img src="images/azure15.png" width=1000>

2. Ấn mũi tên $\text{V}$ cạnh **Debug** -> **Use data flow debug session** -> **Continue with debug cluster**

3. Trong tab Output -> Đưa chuột vào Activity name đang chạy -> Ấn vào icon chiếc kính để xem chi tiết

   <img src="images/azure16.png" width=1500>

   Có thể xem kĩ ở Stages và Lineage

   <img src="images/azure17.png" width=1500>

4. Nếu ổn rồi thì Publish pipeline và Trigger nó

5. Vào Synapse Studio chạy SQL để kiểm tra
   ```sql
   SELECT TOP 100 * FROM SalesOrderHeader
   ```
   ```sql
   SELECT TOP 100 * FROM Customer
   ```

Hiện các pipelines đều chạy thành công nhưng có thể lúc Debug lại báo lỗi "Only one valid authentication should be used for ls_synapse", và ko thể Preview data khi tạo Pipeline. Khắc phục bằng cách thử bật lại **Degbug**, ấn **Validate**, hoặc sửa lại Linked Service `ls_synapse` dùng version `1.0`.

## 5. Using External Compute
1. Vào **Synapse Studio** -> **Data** -> **Linked** -> **Azure Data Lake Storage Gen2** -> **Upload** -> Up 1 file csv bất kì lên, VD `riders.csv`

   <img src="images/azure18.png" with=1500>

2. Ấn chuột phải vào file đó -> **New notebook** -> **Load to DataFrame**. Sẽ tạo ra 1 Notebook mới

   <img src="images/azure19.png" with=500>

3. Vào **Manage** -> **Apache Spark pools** -> **New** -> **Review + create** -> **Create**

4. Vào **Develop** -> Vào Notebook vừa tạo -> Chọn Spark pool vừa tạo -> Run 1 cell \
   Nếu có lỗi thì tăng **Node size** và **Number of node** trong Spark pool (Hiện vẫn đang có lỗi).

5. Tạo 1 cell nữa, điền tên table và chạy
   ```python
   spark.sql("CREATE DATABASE IF NOT EXISTS riders")
   df.write.mode("overwrite").saveAsTable("riders")
   ```
   Sẽ tạo ra 1 table

6. Tạo 1 cell nữa để kiểm tra:
   ```python
   df = spark.sql("SELECT * FROM riders")
   display(df.limit(10))
   ```

   <img src="images/azure20.png" width=1500>

   Nếu ổn rồi thì **Publish all** -> **Publish**

7. Vào **Integrate** -> **New pipeline** -> Kéo thả Notebook vào -> Vào tab Settings -> Chọn Notebook và Spark pool vừa tạo -> **Validate** -> **Publish all** -> **Publish** -> **Add trigger** -> **Trigger now**

   <img src="images/azure21.png" width=1500>

## 6. Power Query
1. Vào **Author** -> **Power Query** -> **New power query**-> Chọn dataset `ds_sqldb_salesorderheader` \
   Nếu có lỗi "We could not evaluate this query due to invalid or missing credentials." thì sửa lại Linked service `ls_sql_sales` dùng Version `1.0`.

   <img src="images/azure22.png" width=1500>

2. Chọn row `SalesOrderID` -> Ấn **Filter rows** -> Set như dưới -> **OK** -> **Publish all** -> **Publish**

   <img src="images/azure23.png" width=1500>

3. Tạo 1 Pipeline mới -> Kéo thả **Power Query** vào -> Vào tab **Settings** -> Chọn Power Query vừa tạo, Tạo Staging linked service mới rồi chọn folder như dưới.

   <img src="images/azure24.png" width=700>

4. Vào tab **Sink** -> Chọn Sink là `ds_synapse_salesaggregatetable`

   <img src="images/azure25.png" width=700>

5. Ấn **Validate** -> **Publish all** -> **Publish** -> **Add trigger** -> **Trigger now**